In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import zipfile
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set()

sns.set_context("talk")
import re

# Ensure that Pandas shows at least 100 characters in columns
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

from pathlib import Path

# SQL
import sqlalchemy

In [2]:
# CREATE TABLE alameda AS SELECT * FROM main_bldg WHERE County='ALAMEDA';
sqlite_uri = "sqlite:///ZCountyExtracts.sqlite"
zce_engine = sqlalchemy.create_engine(sqlite_uri)

In [3]:
zce_engine.table_names()

['alameda']

In [6]:
# change!
sql_expr = """
SELECT * FROM alameda
WHERE PropertyCity == 'BERKELEY'
LIMIT 1000;
"""
berkeley = pd.read_sql(sql_expr, zce_engine)
berkeley.head()

,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,ParcelSequenceNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,RowID2,NoOfUnits,OccupancyStatusStndCode,PropertyCountyLandUseDescription,PropertyCountyLandUseCode,PropertyLandUseStndCode,PropertyStateLandUseDescription,PropertyStateLandUseCode,BuildingOrImprovementNumber,BuildingClassStndCode,BuildingQualityStndCode,BuildingQualityStndCodeOriginal,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,BathSourceStndCode,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FirePlaceTypeStndCode,FireplaceNumber,WaterStndCode,SewerStndCode,MortgageLenderName,TimeshareStndCode,Comments,StoryTypeStndCode
0,28969B05-2AD5-E511-80C1-3863BB43AC67,8925050,06001,CA,ALAMEDA,,082010,18,BKF,16-1410-1-1,,1,,,,,P,3101,,,BENVENUE,AVE,,3101 BENVENUE AVE,BERKELEY,CA,94705,,,,,,,,,,,,,13-000,,,,,,,,,,,,,,,,,,,,,,,,,1,0.038,1639,,,,,339101204,Y,,,C021,Y,37.8536,-122.254,060014239.021007,,,,,,,,0,330739,-1,28969B05-2AD5-E511-80C1-3863BB43AC67,,O,SINGLE FAMILY RESIDENCE,1100,RR101,,,1,,,,,,1914,1921,,2,,,,,,,,2.5,S,,,,,,,,,,,,,,,,
1,29969B05-2AD5-E511-80C1-3863BB43AC67,8925051,06001,CA,ALAMEDA,,082010,18,BKF,16-1410-1-2,,1,,,,,,3101,,,BENVENUE,AVE,,3101 BENVENUE AVE,BERKELEY,CA,94705,,,,,,,,,,,,,17-003,,,,,,,,,,,,,,,,,,,,,,,,,1,0.075,3251,,,,,339101205,Y,,,C021,Y,37.8536,-122.254,060014239.021007,,,,,,,,0,330739,-1,29969B05-2AD5-E511-80C1-3863BB43AC67,,O,SINGLE FAMILY RESIDENCE,1100,RR101,,,1,,,,,,1914,1921,,2,,,,,,,,2.5,S,,,,,,,,,,,,,,,,
2,34969B05-2AD5-E511-80C1-3863BB43AC67,8925062,06001,CA,ALAMEDA,,082010,18,BKF,16-1410-2-1,,1,,,,,P,2634,,,WOOLSEY,ST,,2634 WOOLSEY ST,BERKELEY,CA,94705,,,,,,,,,,,,,13-000,,,,,,,,,,,,,,,,,,,,,,,,,1,0.015,644,,,,,339101206,Y,,,C021,Y,37.8537,-122.254,060014239.021007,,,,,,,,0,330739,-1,34969B05-2AD5-E511-80C1-3863BB43AC67,,O,SINGLE FAMILY RESIDENCE,1100,RR101,,,1,,,,,,1928,1928,,2,6,3,,,,,,2,S,,,,,,,,,,,,,,,,
3,35969B05-2AD5-E511-80C1-3863BB43AC67,8925063,06001,CA,ALAMEDA,,082010,18,BKF,16-1410-2-2,,1,,,,,,2634,,,WOOLSEY,ST,,2634 WOOLSEY ST,BERKELEY,CA,94705,,,,,,,,,,,,,17-003,,,,,,,,,,,,,,,,,,,,,,,,,1,0.033,1436,,,,,339101207,Y,,,C021,Y,37.

In [7]:
# Filter for Berkeley JCE only and not subsidized
#alameda = alameda[alameda['Subsidized'] == 'N']

In [8]:
# Check structure 
# (number of records, number of columns)
berkeley.shape

(1000, 133)

In [21]:
data_dict = pd.read_excel('ZAsmt_DataDictionary_2016-01.xlsx', sheet_name='LandUse', header=5, usecols=np.arange(4))
data_dict.head(2)

,StndCode,PropertyLandUse Description,Prefix Code Classification,Classification
0,AG000,AGRICULTURAL GENERAL,AG,Agricultural
1,AG101,FARM (IRRIGATED OR DRY),AG,Agricultural


In [27]:
residential = data_dict[data_dict['Prefix Code Classification'].isin(['RI', 'RR'])]
R = residential['StndCode']

In [28]:
# Quoted comments refer to the lines in the JC and RC inventory. Specifically each coverage/exemption.

# "Units owned by a government agency"
# "Nursing or hospital units or home for the aged and the like"
# Filter rows to only include residential buildings
print('Current number:', len(berkeley))
berkeley = berkeley[berkeley['PropertyLandUseStndCode'].isin(R)]
print('Current number:', len(berkeley))

1000
930


In [47]:
# "Two unit properties where one unit is owner-occupied"
# Filter out properties with 2 or more units and owner occupied
berkeley = berkeley[~(berkeley['OccupancyStatusStndCode'].isin(['O', 'I']) & (berkeley.NoOfUnits == 2))]
print('Current number:', len(berkeley))

Current number: 916
